<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Pablo Pincheira 
- Nombre de alumno 2: Geraldine Arce


### **Link de repositorio de GitHub:** `https://github.com/geriarri/Laboratorios`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
!pip install praw
!pip install line_profiler
!pip install memory_profiler
!pip install numba

%load_ext autoreload
%autoreload 2
%load_ext line_profiler
%load_ext memory_profiler

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada línea de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [3]:
for it, top_n in enumerate(range(50, 1000, 50)):
    
    #enumerate(range(50, len(top_submissions), 50))
    print("it: ",it)
    print("top_n",top_n)

it:  0
top_n 50
it:  1
top_n 100
it:  2
top_n 150
it:  3
top_n 200
it:  4
top_n 250
it:  5
top_n 300
it:  6
top_n 350
it:  7
top_n 400
it:  8
top_n 450
it:  9
top_n 500
it:  10
top_n 550
it:  11
top_n 600
it:  12
top_n 650
it:  13
top_n 700
it:  14
top_n 750
it:  15
top_n 800
it:  16
top_n 850
it:  17
top_n 900
it:  18
top_n 950


In [4]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {} #3 diccionarios. votos, posts y urls
    top_submissions = list(subreddit.hot(limit=n_hot)) # se almacenan las submissions según el límite dado
    for it, top_n in enumerate(range(50, len(top_submissions), 50)): #it: contador| top_n: 50
        top_n_submissions = top_submissions[:top_n] #revisa las 50, 100, 150, 200, 250...
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions: #se revisa cada submission del filtro hecho
            try:
                ratio = submission.upvote_ratio #parámetro de la submission
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5 #si ratio!= 0.5 se aplica la fó
                    else round(submission.score / 2) #en caso contrario se redondea la división de 2
                )
                upvotes.append(ups) #se añade el elemento ups
                downvotes.append(ups - submission.score) # se añade el elemento down
                post[it].append(submission.title) #se añade el titulo del post
                url[it].append(submission.url) #se añade la url
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T #se añade la tupla/lista de upvotes,downvotes a votes
    return votes, post, url 

In [6]:
votes, post, url = praw_reddit()

In [5]:
%lprun -f praw_reddit praw_reddit()

In [7]:
%%file praw_module.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {} #3 diccionarios. votos, posts y urls
    top_submissions = list(subreddit.hot(limit=n_hot)) # se almacenan las submissions según el límite dado
    for it, top_n in enumerate(range(50, len(top_submissions), 50)): #it: contador| top_n: 50
        top_n_submissions = top_submissions[:top_n] #revisa las 50, 100, 150, 200, 250...
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions: #se revisa cada submission del filtro hecho
            try:
                ratio = submission.upvote_ratio #parámetro de la submission
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5 #si ratio!= 0.5 se aplica la fó
                    else round(submission.score / 2) #en caso contrario se redondea la división de 2
                )
                upvotes.append(ups) #se añade el elemento ups
                downvotes.append(ups - submission.score) # se añade el elemento down
                post[it].append(submission.title) #se añade el titulo del post
                url[it].append(submission.url) #se añade la url
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T #se añade la tupla/lista de upvotes,downvotes a votes
    return votes, post, url 

Overwriting praw_module.py


In [8]:
from praw_module import praw_reddit
%mprun -f praw_reddit praw_reddit()

**Respuesta:**

`Al analizar los resultados del perfilador, se obtiene que el 99,3% del tiempo se destina a la línea de la función "top_submissions = list(subreddit.hot(limit=n_hot))".
Con respecto a la memoria, para la función se utilizan 144 Mb y al llegar a la línea del código mencionada previamente, se evidencia un aumento de 13.8 Mb en la memoria utilizada.
Los resultados antes mencionados destacan con respecto al resto, considerando los otros valores despreciables. Esa parte del código se asocia a la acción de buscar y almacenar las submission asociadas al límite entregado, 1000 en este caso, tiene sentido que sea la parte del código que ocupe más tiempo y memoria. En base al estudio del código se identifican otras partes donde introducir mejoras de eficiencia, pero al analizar los resultados del perfilador se nota que a priori no son cambios que generen una mejora significativa, aunque es algo que debe ser determinado experimentalmente.`

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

1) La memmoria caché permite guardar temporalmente los resultados de operaciones intensivas en un espacio de memoria este espacio puede estar ubicado en memoria (RAM), disco o almacenada de manera remota, esto con el objetivo de mejorar la velocidad y el rendimiento en la ejecución de procesos.

LRU, por sus siglas en inglés Least Recently Used, es un algoritmo optimiza la gestión de la información almacenada en la memoria caché, descartando primero los elementos menos usados recientemente, el algoritmo lleva el seguimiento de lo que se va usando.

2) Se aumenta el consumo de memoria, si esta memoria esta localizada en disco, el acceso puede ser muy lento y el rendimiento puede decaer drásticamente. \
Si se almacena información en caché y ocurre un cambio en la fuente, no se refleja si no se realiza una actualización de este caché, si esto se lleva a un contexto más dinámico o en tiempo real, como un sitio web, puede tener consecuencias y es importante poder discriminar con cuál información sí se puede aplicar caching, reduciendo eventuales consecuencias.  

3) En la función anterior se ocuparía caching para almacenar la información obtenida en reddit, algunas cosencuencia que se pueden considerar son, tener suficiente espacio caché para almacenar toda la información, y lo que se mencionó en el punto 2. 

In [170]:
def praw_reddit_lru(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {} #3 diccionarios. votos, posts y urls
    top_submissions = list(subreddit.hot(limit=n_hot)) # se almacenan las submissions según el límite dado
    for it, top_n in enumerate(range(50, len(top_submissions), 50)): #it: contador| top_n: 50
        top_n_submissions = top_submissions[:top_n] #revisa las 50, 100, 150, 200, 250...
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions: #se revisa cada submission del filtro hecho
            try:
                ratio = submission.upvote_ratio #parámetro de la submission
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5 #si ratio!= 0.5 se aplica la fó
                    else round(submission.score / 2) #en caso contrario se redondea la división de 2
                )
                upvotes.append(ups) #se añade el elemento ups
                downvotes.append(ups - submission.score) # se añade el elemento down
                post[it].append(submission.title) #se añade el titulo del post
                url[it].append(submission.url) #se añade la url
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T #se añade la tupla/lista de upvotes,downvotes a votes
    return votes, post, url

In [171]:
%%timeit
votes, post, url = praw_reddit_lru()

18.6 s ± 511 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [172]:
from functools import lru_cache
@lru_cache(maxsize=1000)
def praw_reddit_lru(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {} #3 diccionarios. votos, posts y urls
    top_submissions = list(subreddit.hot(limit=n_hot)) # se almacenan las submissions según el límite dado
    for it, top_n in enumerate(range(50, len(top_submissions), 50)): #it: contador| top_n: 50
        top_n_submissions = top_submissions[:top_n] #revisa las 50, 100, 150, 200, 250...
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions: #se revisa cada submission del filtro hecho
            try:
                ratio = submission.upvote_ratio #parámetro de la submission
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5 #si ratio!= 0.5 se aplica la fó
                    else round(submission.score / 2) #en caso contrario se redondea la división de 2
                )
                upvotes.append(ups) #se añade el elemento ups
                downvotes.append(ups - submission.score) # se añade el elemento down
                post[it].append(submission.title) #se añade el titulo del post
                url[it].append(submission.url) #se añade la url
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T #se añade la tupla/lista de upvotes,downvotes a votes
    return votes, post, url

In [173]:
%%timeit
votes, post, url = praw_reddit_lru()

The slowest run took 11.00 times longer than the fastest. This could mean that an intermediate result is being cached.
829 ns ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Se nota una diferencia en los tiempos de ejecución sin caché y con caché, esto se debe a que, como se vió antes, la línea del código que más tiempo utiliza es la extracción de información desde reddit, si esta se almacenó en caché, entonces el tiempo total se reduce pues el acceso a la información en dicho caché ocurre de manera más rápida. Se pasó de 18.5 micro segundos por iteración, a 829 nano segundos por iteración (media).

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [49]:
votes[0][:,0] 

array([108, 185, 152,  40, 339,  44, 126,  26, 942,  75,  34, 224, 169,
        15,  12,  19,  19,  19,  28,  10, 186,  10, 690, 342, 306,  87,
        89, 488,  85, 100,  56,  22,  40,  58,  30, 142,  84,   6,  59,
        39,  36, 269,  29, 313,  32,   1,  79,   0,   8, 130])

In [48]:
votes[0] #votes[0][:,0] -> votos positivos

array([[108,  92],
       [185,  18],
       [152,   6],
       [ 40,   1],
       [339,  18],
       [ 44,   6],
       [126,  16],
       [ 26,   0],
       [942,  19],
       [ 75,  20],
       [ 34,   2],
       [224,   7],
       [169,  11],
       [ 15,   0],
       [ 12,   1],
       [ 19,   2],
       [ 19,   4],
       [ 19,   3],
       [ 28,  17],
       [ 10,   1],
       [186,  10],
       [ 10,   2],
       [690,   7],
       [342,  42],
       [306,  30],
       [ 87,   5],
       [ 89,  10],
       [488,  31],
       [ 85,   1],
       [100,  37],
       [ 56,  14],
       [ 22,   6],
       [ 40,   5],
       [ 58,  17],
       [ 30,  20],
       [142,   6],
       [ 84,   6],
       [  6,   2],
       [ 59,   3],
       [ 39,   5],
       [ 36,  22],
       [269,  27],
       [ 29,   7],
       [313,  20],
       [ 32,   9],
       [  1,   0],
       [ 79,  11],
       [  0,   0],
       [  8,   5],
       [130,  14]])

In [54]:
def f1_python(votes_i):
    #votes_i: lista de tuplas. Un elemento de los datos votes.
    #return: sigma: lista de valores sigma. mu: lista de valores mu
    sigma = []
    mu = []
    for k in range(len(votes_i)):
        a = votes_i[k][0] + 1
        b = votes_i[k][1] + 1
        sigma.append(1.65*((a*b)/(((a+b)**2)*(a+b+1)))**(1/2))
        mu.append(a/(a+b))
    return sigma,mu             

In [55]:
def f2_numpy(votes_i):
    #votes_i: lista de tuplas. Un elemento de los datos votes.
    #return: sigma: lista de valores sigma. mu: lista de valores mu
    a = np.array(votes_i[:,0]) + 1 #arreglo de valores a (u+1) 
    b = np.array(votes_i[:,1]) + 1 #arreglo de valores b (d+1)
    sigma = 1.65*((a*b)/(((a+b)**2)*(a+b+1)))**(1/2)
    mu = (a/(a+b))
    return sigma, mu

In [44]:
sigma1, mu1 = f1_python(votes[0])
sigma2, mu2 = f2_numpy(votes[0])

In [46]:
print(sigma1)
print("")
print(sigma2)

[0.05772172203020227, 0.03333724797922435, 0.02659780262765263, 0.05238368866632979, 0.01946948681037983, 0.07735670142483472, 0.04421440369126272, 0.05686025407979019, 0.007578617429732253, 0.06864606000445399, 0.07124636207025818, 0.019640672183940517, 0.030269280302455914, 0.09150793638884733, 0.14022303662380156, 0.11342952476690926, 0.12943664919120146, 0.12298373876248843, 0.11577124910186339, 0.15910649713403296, 0.026792260772424546, 0.17481039291011202, 0.0066334852171823335, 0.02638897033056651, 0.02586529461184104, 0.041381978357230974, 0.05089558163500953, 0.017339652440260443, 0.026065766002620578, 0.062152358188911655, 0.07842834942258242, 0.12534146907934238, 0.07947545466982306, 0.07906935135304256, 0.11120726502504585, 0.028321825997180503, 0.045364186314353884, 0.22798026230355994, 0.04953964758344883, 0.0810555588459959, 0.10271468919803621, 0.027841522689593973, 0.10771437477483892, 0.021819458576122705, 0.10508630463761909, 0.3889087296526011, 0.057622256203874914,

In [47]:
print(mu1)
print("")
print(mu2)

[0.5396039603960396, 0.9073170731707317, 0.95625, 0.9534883720930233, 0.947075208913649, 0.8653846153846154, 0.8819444444444444, 0.9642857142857143, 0.9792315680166147, 0.7835051546391752, 0.9210526315789473, 0.9656652360515021, 0.9340659340659341, 0.9411764705882353, 0.8666666666666667, 0.8695652173913043, 0.8, 0.8333333333333334, 0.6170212765957447, 0.8461538461538461, 0.9444444444444444, 0.7857142857142857, 0.9885550786838341, 0.8886010362694301, 0.908284023668639, 0.9361702127659575, 0.8910891089108911, 0.9385796545105566, 0.9772727272727273, 0.7266187050359713, 0.7916666666666666, 0.7666666666666667, 0.8723404255319149, 0.7662337662337663, 0.5961538461538461, 0.9533333333333334, 0.9239130434782609, 0.7, 0.9375, 0.8695652173913043, 0.6166666666666667, 0.9060402684563759, 0.7894736842105263, 0.9373134328358209, 0.7674418604651163, 0.6666666666666666, 0.8695652173913043, 0.5, 0.6, 0.8972602739726028]

[0.53960396 0.90731707 0.95625    0.95348837 0.94707521 0.86538462
 0.88194444 0.96

Se chequea que en ambas funciones se obtienen los mismos resultados, en la primera con valores no redondeados.

## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [126]:
times_f1 = []
times_f2 = []
for k in range(len(votes)):
    t_f1 = %timeit -o f1_python(votes[k])
    t_f2 = %timeit -o f2_numpy(votes[k])
    times_f1.append(np.sum(t_f1.timings))
    times_f2.append(np.sum(t_f2.timings))

231 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
18.7 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
456 µs ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
18.9 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
702 µs ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
21.4 µs ± 526 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
920 µs ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
21.5 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.15 ms ± 20.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
21.8 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.38 ms ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
24.1 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.62 ms ± 20.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
26.9 µs ± 924 ns per loop (mean ± std. dev. of 7 

In [130]:
@jit(nopython=True)
def f1_python_JIT(votes_i):
    #votes_i: lista de tuplas. Un elemento de los datos votes.
    #return: sigma: lista de valores sigma. mu: lista de valores mu
    sigma = []
    mu = []
    for k in range(len(votes_i)):
        a = votes_i[k][0] + 1
        b = votes_i[k][1] + 1
        sigma.append(1.65*((a*b)/(((a+b)**2)*(a+b+1)))**(1/2))
        mu.append(a/(a+b))
    return sigma,mu  

def f2_numpy_JIT(votes_i):
    #votes_i: lista de tuplas. Un elemento de los datos votes.
    #return: sigma: lista de valores sigma. mu: lista de valores mu
    a = np.array(votes_i[:,0]) + 1 #arreglo de valores a (u+1) 
    b = np.array(votes_i[:,1]) + 1 #arreglo de valores b (d+1)
    sigma = 1.65*((a*b)/(((a+b)**2)*(a+b+1)))**(1/2)
    mu = (a/(a+b))
    return sigma, mu

f1_python_JIT(votes[0])
f2_numpy_JIT(votes[0])

times_f1_jit = [] #python
times_f2_jit = [] #numpy
for k in range(len(votes)):
    t_f1_jit = %timeit -o f1_python_JIT(votes[k])
    t_f2_jit = %timeit -o f2_numpy_JIT(votes[k])
    times_f1_jit.append(np.sum(t_f1_jit.timings))
    times_f2_jit.append(np.sum(t_f2_jit.timings))

4.57 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
17.9 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
8.66 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
19.2 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
12.8 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
21.1 µs ± 374 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
16.1 µs ± 83.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
21.3 µs ± 987 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
20 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
21.7 µs ± 996 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
23.3 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
24.8 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
26.8 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
26.6 µs ± 730 ns per loop (mean ± std. dev

In [131]:
n_datos = np.linspace(50,900,18)
df_plot = pd.DataFrame(n_datos, columns = ['n_datos'])
df_plot['Python'] = times_f1
df_plot['Numpy'] = times_f2
df_plot['Python-JIT'] = times_f1_jit
df_plot['Numpy-JIT'] = times_f2_jit
cols = ["Python", "Numpy", "Python-JIT", "Numpy-JIT"]

In [154]:
import numpy as np
import plotly.express as px

fig = px.line(df_plot,
              x="n_datos",
              y=cols,
              log_y=True)
fig.show()
fig.write_html("medicion_tiempos_results.html")

Sí es posible observar diferencias. Una primera diferencia tiene que ver con los tiempos de ejecución entre las funciones Python y Numpy, siendo los tiempos de la segunda menores. Esta utiliza operadores entre arreglos/vectores sin una iteración sobre cada elemento, a diferencia de la función Python, por lo tanto tiene sentido el resultado, y se puede concluir que el uso de for puede ser menos eficiente.
Por otro lado, la aplicación del compilador sobre las funciones, genera cambios más significativos en Python (diferencia entre línea azul y calipso), pero para Numpy (roja y morada) no se observa una mejora importante. Numba opera paralelizando las funciones, y en la función Numpy no tiene iteraciones que paralelizar pues se calculan los valores en solo una iteración, mediante la operación entre vectores/arrays, a diferencia de lo que ocurre con Python.

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [138]:
votos[:, ]

array([[108,  92],
       [185,  18],
       [152,   6],
       ...,
       [531,  11],
       [222,   0],
       [164,  20]])

In [139]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
#posterior_mean, std_err = intervals_numpy(votos[:, 0], votos[:, 1])
std_err, posterior_mean = f2_numpy(votos[:, ]) #se ajusta el código a la función creada en 1.3
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



In [153]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()
fig.write_html("posts_results.html")

**Respuesta:**

Según está calculada la media, en base a la fórmula entregada, si se tiene un valor cercano a 1, el post tiene mayor cantidad de votos positivos. Si tiene un valor cercano a 0.5, es neutro (misma proporción positivos/negativos) y si es cercano a cero, tiene mayor cantidad de votos negativos)
Se observan algunos posts con contenido que podría catalogarse como negativo, como "Alumna fue apuñalada tras oponerse a continuidad de toma en liceo en Concepción" o "Hoy falleció la profesora Victoria Casto ..." pero tienen una media sobre 0.97, y se esperaría que fuera bajo 0.5 al menos, esto puede dar cuenta que los votos "positivos" o "negativos" que se asocian a los posts no necesariamente están relacionados con la aprobación/desaprobación o like/dislike del contenido, sino más bien por la relevancia de este, por ejemplo. El analisis de realiza en base al poco conocimiento que se tiene sobre la dinámica de la plataforma.


<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>